In [5]:
import imageio
img_arr = imageio.v2.imread('./candy2.jpeg')
img_arr.shape

(725, 1086, 3)

In [12]:
import torch
img_t = torch.from_numpy(img_arr)
img_t = img_t.permute(2,0,1)
img_t.shape

torch.Size([3, 725, 1086])

In [59]:
import os

data_dir = './image-cats/'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png']

batch_size = 3
n_channels = 3
batch = torch.zeros(batch_size, n_channels, 256, 256, dtype = torch.uint8)

for i, name in enumerate(filenames):
    img_arr = imageio.v2.imread(os.path.join(data_dir, name))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2,0,1)
    batch[i] = img_t

batch = batch.float()
batch /= 255
batch


for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:,c] = (batch[:,c] - mean) / std


tensor(1.)
tensor(1.)
tensor(1.)


In [69]:
data_dir = './volumetric-dicom/2-LUNG 3.0  B70f-04083/'
vol_arr = imageio.volread(data_dir, 'DICOM')
vol_arr.shape
vol_t = torch.from_numpy(vol_arr)
batch = vol_t.unsqueeze(0).unsqueeze(0)

Reading DICOM (examining files): 1/99 files (1.0%72/99 files (72.7%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 31/99  (31.378/99  (78.899/99  (100.0%)


In [117]:
import numpy as np

data_path = './tabular-wine/winequality-white.csv'
table_arr = np.loadtxt(data_path, dtype=np.float32, delimiter=';', skiprows=1)
table_t = torch.from_numpy(table_arr)

data_t = table_t[:,:-1]
target_t = table_t[:,-1]
target_t.shape[0]

#1 : (1,0,0,0,0,0,0,0,0,0)
#2 : (0,1,0,0,0,0,0,0,0,0) ...

one_hot_target_t = torch.zeros(target_t.shape[0], 10).long()
one_hot_target_t.scatter_(1, target_t.unsqueeze(1).long(), 1.0)

data_mean = torch.mean(data_t, dim=0)
data_std = torch.std(data_t, dim = 0)
data_norm = (data_t - data_mean) / data_std
data_norm.shape

bad_data = data_t[(target_t <= 3)]
medium_data = data_t[(target_t > 3) & (target_t < 7)]
good_data = data_t[target_t >= 7]
good_data.shape

torch.Size([1060, 11])